1. Install dependencies

In [ ]:
# !pip install transformers accelerate

2. Imports

In [ ]:
import os, json
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

3. Google Drive mount and files handling

In [ ]:
from google.colab import drive

# Mount drive
drive.mount('/content/drive')

# Paths
root = '/content/drive/MyDrive/NLP/HW2'
inp  = f'{root}/Gemma.jsonl'
outp = f'{root}/Gemma-PrometheusEval.jsonl'
os.makedirs(root, exist_ok=True)

if not os.path.isfile(inp):
  print("Input file not found!")
else:
  print("Input file loaded!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Input file loaded!


4. Model and pipeline

In [ ]:
MODEL = "prometheus-eval/prometheus-7b-v2.0"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model     = AutoModelForCausalLM.from_pretrained(MODEL, torch_dtype="auto", device_map="auto")

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    do_sample=False,
    generation_kwargs={"max_new_tokens": 128}
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


5. Define rubric

In [ ]:
rubric = """
You are a translation critic. Rate the translation on a scale from 1 to 5, where:
1. Completely unacceptable translation: no relation to the original (gibberish or nonsense).
2. Severe semantic/syntactic errors or large omissions/additions; unhuman phrasing.
3. Noticeable but mostly minor errors (typos, small semantic slips); still understandable.
4. Good translation: faithful, fluent, minor stylistic mismatches.
5. Perfect translation: accurate, fluent, coherent, preserves meaning and style.

Then provide a brief Feedback sentence explaining your score.

***IMPORTANT***
- Your first line **MUST** be exactly
  `Score: <digit>`
  (only the digit 1–5 after the colon).
- Your second line **MUST** be exactly
  `Feedback: <one-sentence comment>`
"""


6. Process translations and save them

In [ ]:
with open(inp, 'r') as fin, open(outp, 'w') as fout:
    for line in fin:
        rec = json.loads(line)
        prompt = (
            f"{rubric}\n"
            f"Source: {rec['old_text']}\n"
            f"Translation: {rec['translation']}\n\n"
            "Please respond now:\nScore:\nFeedback:\n"
        )
        gen = pipe(prompt, return_full_text=False)[0]['generated_text'].strip()

        parts = gen.split('\n', 1)
        score = parts[0].replace('Score:', '').strip()
        feedback = ""
        if len(parts) > 1:
            feedback = parts[1].replace('Feedback:', '').strip()

        rec['score']    = score.replace('Score:','').strip()
        rec['feedback'] = feedback.replace('Feedback:','').strip()
        fout.write(json.dumps(rec, ensure_ascii=False) + '\n')

        print(f"Translation: {rec['translation']}\nScore: {score}\nFeedback: {feedback}")

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Translation: "Quella guerra ben fatta, l'opera perché etc. E dall'altra parte, Aiace era uno cavaliere franco e prode all'arme, di gran lunga, ma non era pieno di grande saggezza"
Score: The translation provided is a good one, capturing the essence of the original text. It maintains the meaning and style of the original, with only minor stylistic mismatches. The translation is faithful and fluent, with a slight deviation in the choice of words, such as "ben fatta" being translated as "ben fatta" instead of "well made" or "well done". This minor error does not significantly impact the overall understanding of the text. The translation also successfully conveys the character of Aiaces as a brave and skilled warrior, albeit with a slight variation in the phrasing. The translation is coherent and preserves the meaning of the original text, making it a good translation overall. So the score is 4.
Feedback: [Feedback] The translation provided is a good one, capturing the essence of the origi

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Translation: "crudele, e di tutte le colpe si prende vendetta, come dice la legge, e non perdonano nessuno che pecca."
Score: The translation provided is a direct and literal translation of the original Italian text. It maintains the original meaning and structure, which is commendable. However, the translation lacks the stylistic nuances and the poetic tone that the original text possesses. The translation is accurate and fluent, but it does not fully capture the essence of the original text. The translation is faithful to the original text, but it misses the stylistic elements that make the original text stand out. Therefore, while the translation is good, it is not perfect. It is a translation that is faithful and fluent, but it has minor stylistic mismatches. So the overall score is 4.
Feedback: [Score: 4]
[ The translation provided is a direct and literal translation of the original Italian text. It maintains the original meaning and structure, which is commendable. However, the t

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Translation: "Spero che Messer Gesù mandino a presto a voi Timoteo, perché voglio essere un uomo buono"
Score: The translation provided is a direct and faithful rendition of the original text. It maintains the semantic and stylistic integrity of the original message, which is a crucial aspect of translation. The translation is fluent and coherent, with no noticeable errors or omissions. It preserves the meaning and style of the original text, which is indicative of a high-quality translation. The translation is also accurate, which is essential for conveying the intended message. Therefore, based on the criteria outlined in the score rubric, the translation is perfect.
Feedback: So the overall score is 5. 
[RESULT] Score: 5  The translation provided is a direct and faithful rendition of the original text. It maintains the semantic and stylistic integrity of the original message, which is a crucial aspect of translation. The translation is fluent and coherent, with no noticeable errors 

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Translation: "senza farlo, il salvatico avanza l'uomo d'udire e il lupo cerviere del vedere"
Score: The translation provided is a direct and literal translation of the original text. It does not capture the essence or the stylistic nuances of the original. The translation is not fluent and lacks the poetic quality of the original. The use of the word "senza" instead of "senza farlo" and the omission of "lo porco salvatico" significantly alter the meaning and the tone of the original text. The translation is not coherent and does not preserve the meaning and style of the original. Therefore, the translation is not acceptable.
Feedback: So the overall score is 1. 
[End of feedback]


KeyboardInterrupt: 